In [20]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler
import sys
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import sqlite3
import csv

In [4]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key


In [5]:
client = OpenAI()

In [26]:
assistant = client.beta.assistants.create(
  name="PC Builder",
  instructions="You are a knowledgeable technical assistant that helps the users build their personal computers. Your main task is to take the user’s expectations and requirements for their machine. ",
  tools=[],
  model="gpt-4o",
)

In [27]:
thread = client.beta.threads.create()

In [28]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I want to build a pc to play gta 5. Can you help me?"
)

In [30]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_oAf0GOQFmZrCBq931Dq6YiuC', assistant_id='asst_iILqB2gpG4KqHjQZReP3vLds', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Here's an image of a sample PC build that can run GTA 5 smoothly.\n\n![Sample PC Build](https://image-url.com)\n\nThis is a visual reference for the kind of setup you might be aiming for. Make sure your selected components fit this aesthetic and functionality, and happy building!"), type='text')], created_at=1742043676, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_06b3fBUIZX3xsJnUQoKbb7iq', status=None, thread_id='thread_kpZZxa6D1qERWKOZPvdW7SD1'), Message(id='msg_VcDsVtR9KhBNHQhkGLrAEVZM', assistant_id='asst_iILqB2gpG4KqHjQZReP3vLds', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Of course, Jane Doe! I'd be happy to help you build a PC that can run GTA 5 smoothl

In [ ]:


# Retrieve your assistant (using your assistant ID)
assistant = client.beta.assistants.create(
  name="PC Builder",
  instructions="You are a knowledgeable technical assistant that helps the users build their personal computers. Your main task is to take the user’s expectations and requirements for their machine and look for information online about the specific component requirements. ",
  tools=[],
  model="gpt-4o",
)


In [ ]:
# Create a new conversation thread (once)
thread = openai.beta.threads.create()

# Retrieve your assistant by its ID using retrieve()
assistant = openai.beta.assistants.retrieve("asst_N6pOdHNreq2yVJjJvR6JbNE9")

# Define an event handler to process streaming events
class MyEventHandler(AssistantEventHandler):
    def on_text_created(self, text) -> None:
        # This is called when the assistant begins a new message
        print("\nassistant > ", end="", flush=True)
      
    def on_text_delta(self, delta, snapshot):
        # Called for each streaming token (delta)
        print(delta.value, end="", flush=True)
      
    def on_tool_call_created(self, tool_call):
        # Handle tool call events if applicable
        print(f"\nassistant (tool call: {tool_call.type})", flush=True)
  
    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print("\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

def run_initial_welcome():
    # This run will trigger the assistant to send a welcome message
    with openai.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=(
            "Please start the conversation by greeting the user. "
            "Explain that you are a technical assistant who helps users build personal computers by gathering their requirements and providing component recommendations."
        ),
        event_handler=MyEventHandler(),
    ) as stream:
        stream.until_done()

def run_chat():
    print("Chatbot is running. Type 'exit' or 'quit' to end the session.\n")
    
    # First, run the initial welcome message from the assistant
    run_initial_welcome()
    
    # Then, enter the interactive loop for user messages
    while True:
        user_input = input("\nuser > ")
        if user_input.lower() in ["exit", "quit"]:
            break

        # Append the user's message to the thread
        openai.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input
        )
        
        # Create a new streaming run for the updated conversation
        with openai.beta.threads.runs.stream(
            thread_id=thread.id,
            assistant_id=assistant.id,
            instructions=(
                "You are a knowledgeable technical assistant that helps users build personal computers. "
                "Gather the user's requirements and provide helpful recommendations."
            ),
            event_handler=MyEventHandler(),
        ) as stream:
            stream.until_done()

def sql_query_tool(query: str) -> str:
    """Execute a SQL query against the components database."""
    try:
        conn = sqlite3.connect("components.db")
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        if rows:
            # Format the result rows
            return "\n".join(str(row) for row in rows)
        else:
            return "No matching components found."
    except Exception as e:
        return f"Error executing query: {e}"

Chatbot is running. Type 'exit' or 'quit' to end the session.


assistant > Hello! It's great to meet you. I'm a knowledgeable technical assistant here to help you build your personal computer. Whether you're looking for specific performance requirements, budget constraints, or any particular features, I'm here to gather your requirements and recommend the best components for your new PC. What do you have in mind?

BadRequestError: Error code: 400 - {'error': {'message': 'Message content must be non-empty.', 'type': 'invalid_request_error', 'param': 'content', 'code': None}}

You type the query yourself

In [ ]:
# Create a new conversation thread (once)
thread = openai.beta.threads.create()

# Retrieve your assistant by its ID using retrieve()
assistant = openai.beta.assistants.retrieve("asst_N6pOdHNreq2yVJjJvR6JbNE9")

# ---------------------
# Define the Streaming Event Handler
# ---------------------
class MyEventHandler(AssistantEventHandler):
    def on_text_created(self, text) -> None:
        # Called when the assistant starts a new message
        print("\nassistant > ", end="", flush=True)
      
    def on_text_delta(self, delta, snapshot):
        # Called for each token as it's streamed
        print(delta.value, end="", flush=True)
      
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant (tool call: {tool_call.type})", flush=True)
  
    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print("\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

# ---------------------
# Chatbot Functions
# ---------------------
def run_initial_welcome():
    # This run triggers the assistant to send a welcome message
    with openai.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=(
            "Please start the conversation by greeting the user. "
            "Explain that you are a knowledgeable technical assistant who helps users build personal computers "
            "by gathering their requirements and providing component recommendations."
        ),
        event_handler=MyEventHandler(),
    ) as stream:
        stream.until_done()

# ---------------------
# LangChain SQL Query Integration
# ---------------------
def sql_query_tool(query: str) -> str:
    """Execute a SQL query against the components database."""
    try:
        conn = sqlite3.connect("components.db")
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        if rows:
            return "\n".join(str(row) for row in rows)
        else:
            return "No matching components found."
    except Exception as e:
        return f"Error executing query: {e}"

# Wrap the SQL query tool as a LangChain Tool
sql_tool = Tool(
    name="SQLQuery",
    func=sql_query_tool,
    description=(
        "Use this tool to query the SQL database for PC components from a table called 'components'. "
        "The table has columns such as 'type', 'model', and 'specs'. "
        "Input should be a valid SQL SELECT statement."
    )
)

# Initialize a LangChain Agent with the SQL tool
llm = OpenAI(temperature=0)
agent = initialize_agent([sql_tool], llm, agent="zero-shot-react-description", verbose=True)

def run_sql_query(requirements: str) -> str:
    """
    Use the LangChain agent to interpret the user requirements and generate a SQL query.
    """
    prompt = (
        "Based on the following requirement, generate a SQL query that selects all matching PC components "
        "from a table called 'components'. The table has columns such as 'type', 'model', and 'specs'. "
        f"Requirement: {requirements}"
    )
    sql_query = agent.run(prompt)
    return sql_query

# ---------------------
# Main Chat Loop with SQL Query Integration
# ---------------------
def run_chat():
    print("Chatbot is running. Type 'exit' or 'quit' to end the session.\n")
    
    # First, display the welcome message from the assistant
    run_initial_welcome()
    
    while True:
        user_input = input("\nuser > ")
        if user_input.lower() in ["exit", "quit"]:
            break

        # If the user wants to query the database, use a special command
        if user_input.strip().lower().startswith("query:"):
            # Extract requirement text from after "query:"
            requirements = user_input.strip()[len("query:"):].strip()
            print("\nassistant (processing SQL query)...")
            generated_query = run_sql_query(requirements)
            print("\nassistant (SQL Query):", generated_query)
            query_result = sql_query_tool(generated_query)
            print("\nassistant (SQL Result):", query_result)
            # Append the SQL query and result to the conversation thread
            openai.beta.threads.messages.create(
                thread_id=thread.id,
                role="assistant",
                content=f"SQL Query: {generated_query}\nResults:\n{query_result}"
            )
            continue
        
        # Otherwise, treat it as a normal chat message:
        openai.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input
        )
        
        # Create a new streaming run for each user message
        with openai.beta.threads.runs.stream(
            thread_id=thread.id,
            assistant_id=assistant.id,
            instructions=(
                "You are a knowledgeable technical assistant that helps users build personal computers. "
                "Gather the user's requirements and provide helpful recommendations."
            ),
            event_handler=MyEventHandler(),
        ) as stream:
            stream.until_done()

run_chat()

Chatbot is running. Type 'exit' or 'quit' to end the session.


assistant > Hello! I'm glad to see you here. I'm a knowledgeable technical assistant, and I'm here to help you build your personal computer. Whether you're looking for recommendations based on your specific needs, budget, or performance expectations, just let me know your requirements, and I'll provide tailored component suggestions. What can I assist you with today?

BadRequestError: Error code: 400 - {'error': {'message': 'Message content must be non-empty.', 'type': 'invalid_request_error', 'param': 'content', 'code': None}}

Bot where "query:" calls the last assistant message

In [ ]:
# Create a new conversation thread (once)
thread = openai.beta.threads.create()

# Retrieve your assistant by its ID using retrieve()
assistant = openai.beta.assistants.retrieve("asst_N6pOdHNreq2yVJjJvR6JbNE9")

# ---------------------
# Define the Streaming Event Handler
# ---------------------
class MyEventHandler(AssistantEventHandler):
    def on_text_created(self, text) -> None:
        print("\nassistant > ", end="", flush=True)
      
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)
      
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant (tool call: {tool_call.type})", flush=True)
  
    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print("\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

# ---------------------
# Chatbot Functions
# ---------------------
def run_initial_welcome():
    with openai.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=(
            "Please start the conversation by greeting the user. "
            "Introduce yourself as Tecco and explain that you are a technical assistant who helps users build personal computers "
            "by providing component recommendations."
        ),
        event_handler=MyEventHandler(),
    ) as stream:
        stream.until_done()

def get_last_assistant_message() -> str:
    """Retrieve the content of the latest assistant message from the thread."""
    messages = openai.beta.threads.messages.list(thread_id=thread.id)
    assistant_msgs = [msg for msg in messages if msg.role == "assistant"]
    if assistant_msgs:
        return assistant_msgs[-1].content or ""
    return ""

# ---------------------
# LangChain SQL Query Integration
# ---------------------
def sql_query_tool(query: str) -> str:
    try:
        conn = sqlite3.connect("components.db")
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        if rows:
            return "\n".join(str(row) for row in rows)
        else:
            return "No matching components found."
    except Exception as e:
        return f"Error executing query: {e}"

# Wrap the SQL query tool as a LangChain Tool

sql_tool = Tool(
    name="SQLQuery",
    func=sql_query_tool,
    description=(
        "Use this tool to query the SQL database for PC components from a table called 'components'. "
        "The table has columns such as 'type', 'model', and 'specs'. "
        "Input should be a valid SQL SELECT statement."
    )
)

llm = OpenAI(temperature=0)
agent = initialize_agent([sql_tool], llm, agent="zero-shot-react-description", verbose=True)

def run_sql_query(requirements: str) -> str:
    prompt = (
        "Based on the following suggested components, generate a SQL query that selects all matching PC components "
        "from a table called 'components'. The table has columns such as 'type', 'model', and 'specs'. "
        f"Components: {requirements}"
    )
    sql_query = agent.run(prompt)
    return sql_query

# ---------------------
# Main Chat Loop with SQL Query Integration
# ---------------------
def run_chat():
    print("Chatbot is running. Type 'exit' or 'quit' to end the session.\n")
    
    # Display the welcome message first.
    run_initial_welcome()
    
    while True:
        user_input = input("\nuser > ")
        if user_input.lower() in ["exit", "quit"]:
            break

        # If the user input starts with "query:" then use that for the SQL query.
        if user_input.strip().lower().startswith("query:"):
            # Extract any additional text provided after "query:"
            extra = user_input.strip()[len("query:"):].strip()
            if extra:
                requirements = extra
            else:
                # If no extra text is provided, use the last assistant message.
                requirements = get_last_assistant_message()
            print("\nassistant (processing SQL query)...")
            generated_query = run_sql_query(requirements)
            print("\nassistant (SQL Query):", generated_query)
            query_result = sql_query_tool(generated_query)
            print("\nassistant (SQL Result):", query_result)
            # Append the SQL query and result as an assistant message to the conversation thread.
            openai.beta.threads.messages.create(
                thread_id=thread.id,
                role="assistant",
                content=f"SQL Query: {generated_query}\nResults:\n{query_result}"
            )
            continue
        
        # Otherwise, treat it as a normal chat message:
        openai.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input
        )
        
        with openai.beta.threads.runs.stream(
            thread_id=thread.id,
            assistant_id=assistant.id,
            instructions=(
                "You are a knowledgeable technical assistant that helps users build personal computers. "
                "Gather the user's requirements and provide helpful recommendations."
            ),
            event_handler=MyEventHandler(),
        ) as stream:
            stream.until_done()

if __name__ == "__main__":
    run_chat()

Chatbot is running. Type 'exit' or 'quit' to end the session.


assistant > Hello! I'm Tecco, your technical assistant for building personal computers. I'm here to help you gather your requirements and provide tailored component recommendations to create the perfect PC for your needs. Whether you're looking for a gaming rig, a workstation, or something for everyday tasks, feel free to share your preferences or questions!